<a href="https://colab.research.google.com/github/Noorasyikeen/InvestoInsight/blob/noora_nlp/SP500_yearly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 142.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import string

import pickle

#FinBERT Analysis
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/InvestoInsight/

/content/drive/MyDrive/InvestoInsight


In [ ]:
%ls

Dataset/  SP500_yearly.ipynb


In [ ]:
sp500_year = pd.read_csv('raw_data/SP500.csv', index_col=0)

with open('sp500pickle.pkl','wb') as sp500_pkl:
  pickle.dump(sp500_year,sp500_pkl)


sp500_year

,title,date,stock
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,A
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03,A
2,71 Biggest Movers From Friday,2020-05-26,A
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22,A
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22,A
...,...,...,...
401955,Stocks That Made New 52-Wk Highs Earlier Today...,2018-02-20,ZTS
401956,Zoetis Sees FY18 Adj. EPS $2.96-$3.10 vs $2.98...,2018-02-15,ZTS
401957,Zoetis Reports Q4 Adj. EPS $0.69 vs $0.66 Est....,2018-02-15,ZTS
401958,"Earnings Scheduled For February 15, 2018",2018-02-15,ZTS


In [ ]:
def basic_cleaning(sentence):
    sentence = sentence.lower()

    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '')

    sentence = sentence.strip()

    return sentence

cleaned_title = [basic_cleaning(sentence) for sentence in sp500_year['title']]

sp500_year['title'] = cleaned_title

with open('sp500pickle.pkl','wb') as sp500_pkl:
  pickle.dump(sp500_year,sp500_pkl)

sp500_year

,title,date,stock
0,stocks that hit 52week highs on friday,2020-06-05,A
1,stocks that hit 52week highs on wednesday,2020-06-03,A
2,71 biggest movers from friday,2020-05-26,A
3,46 stocks moving in fridays midday session,2020-05-22,A
4,b of a securities maintains neutral on agilent...,2020-05-22,A
...,...,...,...
401955,stocks that made new 52wk highs earlier today ...,2018-02-20,ZTS
401956,zoetis sees fy18 adj eps 296310 vs 298 est sal...,2018-02-15,ZTS
401957,zoetis reports q4 adj eps 069 vs 066 est sales...,2018-02-15,ZTS
401958,earnings scheduled for february 15 2018,2018-02-15,ZTS


In [ ]:
sp500_year['date'] = pd.to_datetime(sp500_year['date'])

with open('sp500pickle.pkl','wb') as sp500_pkl:
  pickle.dump(sp500_year,sp500_pkl)

## 2018

In [ ]:
#FinBERT Tokenizer and model 2018

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone',model_max_length=512)

In [ ]:
sp500_2018 = sp500_year[sp500_year['date']<pd.to_datetime("2019-01-01")]

sp500_2018

,title,date,stock
157,50 biggest movers from yesterday,2018-11-21,A
158,36 stocks moving in tuesdays midday session,2018-11-20,A
159,shares of agilent technologies are trading up ...,2018-11-20,A
160,morgan stanley maintains overweight on agilent...,2018-11-20,A
161,26 stocks moving in tuesdays premarket session,2018-11-20,A
...,...,...,...
401955,stocks that made new 52wk highs earlier today ...,2018-02-20,ZTS
401956,zoetis sees fy18 adj eps 296310 vs 298 est sal...,2018-02-15,ZTS
401957,zoetis reports q4 adj eps 069 vs 066 est sales...,2018-02-15,ZTS
401958,earnings scheduled for february 15 2018,2018-02-15,ZTS


In [ ]:
headline_list2018 = sp500_2018['title'].tolist()
stock_column2018 = sp500_2018['stock'].tolist()
date_column2018 = sp500_2018['date'].tolist()

In [ ]:
nlp2018 = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

In [ ]:
results2018 = nlp2018(headline_list2018)

In [ ]:
results2018

In [ ]:
sentiment_result = []

def chunk_list(lst, n):
    "Yield successive n-sized chunks from lst."
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


STRIDE = 100

finbert.eval()

n=0
for lines, stocks, dates in zip(chunk_list(headline_list2018, STRIDE), chunk_list(stock_column2018, STRIDE),chunk_list(date_column2018, STRIDE) ):

  input = tokenizer(lines, padding = True, truncation = True,  return_tensors='pt')

  outputs = finbert(**input)

  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)

  print(f"{n+1}/{int(len(headline_list2018)/STRIDE)}")

  for headline, dates, stock, neu, pos, neg in zip(lines, dates, stocks, prediction[:, 0].tolist(), prediction[:, 1].tolist(), prediction[:, 2].tolist() ):
    # sentiment_result = sentiment_result.append({'Headline': headline, 'Date': dates, 'Stock': stock, 'Neu': neu, 'Pos': pos, 'Neg': neg}, ignore_index=True)
    sentiment_result.append([headline, dates, stock, neu, pos, neg])

  n+=1

In [ ]:
sentiment_result2018 = pd.DataFrame(sentiment_result, columns= ['Headline', 'Date', 'Stock', 'Neutral', 'Positive', 'Negative' ])

sentiment_result2018

,Headline,Date,Stock,Neutral,Positive,Negative
0,50 biggest movers from yesterday,2018-11-21,A,8.389207e-01,3.987503e-02,1.212042e-01
1,36 stocks moving in tuesdays midday session,2018-11-20,A,9.999959e-01,1.325494e-07,3.961927e-06
2,shares of agilent technologies are trading up ...,2018-11-20,A,4.851770e-08,1.000000e+00,1.385383e-09
3,morgan stanley maintains overweight on agilent...,2018-11-20,A,3.539342e-07,9.999996e-01,2.367040e-08
4,26 stocks moving in tuesdays premarket session,2018-11-20,A,9.999979e-01,1.054393e-07,2.009748e-06
...,...,...,...,...,...,...
35445,stocks that made new 52wk highs earlier today ...,2018-02-20,ZTS,9.999713e-01,1.914438e-06,2.687052e-05
35446,zoetis sees fy18 adj eps 296310 vs 298 est sal...,2018-02-15,ZTS,9.999706e-01,9.158387e-06,2.027603e-05
35447,zoetis reports q4 adj eps 069 vs 066 est sales...,2018-02-15,ZTS,9.999902e-01,8.538238e-06,1.209126e-06
35448,earnings scheduled for february 15 2018,2018-02-15,ZTS,9.999971e-01,4.552112e-08,2.895475e-06


In [ ]:
sentiment_result2018.to_csv('Dataset/sentiment2018.csv')

## 2019






### Part 1

In [ ]:
#FinBERT Tokenizer and model 2019 p1

finbert19p1 = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')
tokenizer19p1 = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone',model_max_length=512)

In [ ]:
sp500_2019p1 =sp500_year[(sp500_year['date'] >= pd.to_datetime("2019-01-01")) & (sp500_year['date'] <= pd.to_datetime("2019-06-01"))]

with open('sp500_2019part1.pkl','wb') as sp500_2019_pkl1:
  pickle.dump(sp500_2019p1,sp500_2019_pkl1)

sp500_2019p1

,title,date,stock
124,shares of many healthcare companies are tradin...,2019-05-16,A
125,60 biggest movers from yesterday,2019-05-16,A
126,44 stocks moving in wednesdays midday session,2019-05-15,A
127,benzinga pros top 10 mostsearched tickers for ...,2019-05-15,A
128,bank of america maintains buy on agilent techn...,2019-05-15,A
...,...,...,...
401883,zoetis q4 eps 079 beats 077 estimate sales 16b...,2019-02-14,ZTS
401884,earnings scheduled for february 14 2019,2019-02-14,ZTS
401885,zoetis q4 earnings preview,2019-02-13,ZTS
401886,the week ahead in biotech conferences pdufa da...,2019-02-10,ZTS


In [ ]:
with open('sp500_2019part1.pkl','rb') as loading2:
  sp500_2019p1 = pickle.load(loading2)

In [ ]:
sp500_2019p1

,title,date,stock
124,shares of many healthcare companies are tradin...,2019-05-16,A
125,60 biggest movers from yesterday,2019-05-16,A
126,44 stocks moving in wednesdays midday session,2019-05-15,A
127,benzinga pros top 10 mostsearched tickers for ...,2019-05-15,A
128,bank of america maintains buy on agilent techn...,2019-05-15,A
...,...,...,...
401883,zoetis q4 eps 079 beats 077 estimate sales 16b...,2019-02-14,ZTS
401884,earnings scheduled for february 14 2019,2019-02-14,ZTS
401885,zoetis q4 earnings preview,2019-02-13,ZTS
401886,the week ahead in biotech conferences pdufa da...,2019-02-10,ZTS


In [ ]:
headline_list2019p1 = sp500_2019p1['title'].tolist()
stock_column2019p1 = sp500_2019p1['stock'].tolist()
date_column2019p1 = sp500_2019p1['date'].tolist()

In [ ]:
nlp2019p1 = pipeline("text-classification", model=finbert19p1, tokenizer=tokenizer19p1)

In [ ]:
results2019p1 = nlp2019p1(headline_list2019p1)

with open ('res2019p1.pkl', 'wb') as files:
  pickle.dump(results2019p1,files)

In [ ]:
results2019p1

[{'label': 'Neutral', 'score': 0.996414303779602},
 {'label': 'Neutral', 'score': 0.9972604513168335},
 {'label': 'Neutral', 'score': 0.996414303779602},
 {'label': 'Neutral', 'score': 0.9981935620307922},
 {'label': 'Neutral', 'score': 0.998350977897644},
 {'label': 'Neutral', 'score': 0.9980161190032959},
 {'label': 'Negative', 'score': 0.9836779832839966},
 {'label': 'Neutral', 'score': 0.9999904632568359},
 {'label': 'Neutral', 'score': 0.9999836683273315},
 {'label': 'Neutral', 'score': 0.9999849796295166},
 {'label': 'Neutral', 'score': 0.996414303779602},
 {'label': 'Negative', 'score': 0.8813025951385498},
 {'label': 'Positive', 'score': 1.0},
 {'label': 'Neutral', 'score': 0.9999786615371704},
 {'label': 'Neutral', 'score': 0.9999712705612183},
 {'label': 'Positive', 'score': 1.0},
 {'label': 'Neutral', 'score': 0.9999804496765137},
 {'label': 'Neutral', 'score': 0.9999970197677612},
 {'label': 'Positive', 'score': 0.9998825788497925},
 {'label': 'Neutral', 'score': 0.99982833

In [ ]:
with open('res2019p1.pkl','rb') as loading:
  result_2019p1 = pickle.load(loading)

In [ ]:
sentiment_result2019p1 = []

def chunk_list(lst, n):
    "Yield successive n-sized chunks from lst."
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


STRIDE = 70

finbert19p1.eval()

n=0
for lines, stocks, dates in zip(chunk_list(headline_list2019p1, STRIDE), chunk_list(stock_column2019p1, STRIDE),chunk_list(date_column2019p1, STRIDE) ):

  input = tokenizer19p1(lines, padding = True, truncation = True,  return_tensors='pt')

  outputs = finbert19p1(**input)

  with open ("Dataset/sentiment 2019 pickle/f'output_{n+1}.pkl", 'wb') as result:
    pickle.dump(outputs,result)

  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)

  with open ("Dataset/sentiment 2019 pickle/f'pred_{n+1}.pkl", 'wb') as result2:
    pickle.dump(prediction,result2)

  print(f"{n+1}/{int(len(headline_list2019p1)/STRIDE)}")

  for headline, dates, stock, neu, pos, neg in zip(lines, dates, stocks, prediction[:, 0].tolist(), prediction[:, 1].tolist(), prediction[:, 2].tolist() ):
    # sentiment_result = sentiment_result.append({'Headline': headline, 'Date': dates, 'Stock': stock, 'Neu': neu, 'Pos': pos, 'Neg': neg}, ignore_index=True)
    sentiment_result2019p1.append([headline, dates, stock, neu, pos, neg])



  n+=1





In [ ]:
sentiment_result2019p1 = pd.DataFrame(sentiment_result2019p1, columns= ['Headline', 'Date', 'Stock', 'Neutral', 'Positive', 'Negative' ])

sentiment_result2019p1

,Headline,Date,Stock,Neutral,Positive,Negative
0,shares of many healthcare companies are tradin...,2019-05-16,A,8.473804e-03,8.995113e-01,9.201495e-02
1,60 biggest movers from yesterday,2019-05-16,A,9.548358e-01,4.759314e-03,4.040486e-02
2,44 stocks moving in wednesdays midday session,2019-05-15,A,9.999965e-01,1.212762e-07,3.387590e-06
3,benzinga pros top 10 mostsearched tickers for ...,2019-05-15,A,9.999952e-01,1.778685e-06,3.002188e-06
4,bank of america maintains buy on agilent techn...,2019-05-15,A,7.544150e-06,9.998995e-01,9.292139e-05
...,...,...,...,...,...,...
15323,zoetis q4 eps 079 beats 077 estimate sales 16b...,2019-02-14,ZTS,3.400973e-08,1.000000e+00,7.922034e-09
15324,earnings scheduled for february 14 2019,2019-02-14,ZTS,9.999968e-01,9.874385e-08,3.078802e-06
15325,zoetis q4 earnings preview,2019-02-13,ZTS,9.998604e-01,9.476034e-05,4.476140e-05
15326,the week ahead in biotech conferences pdufa da...,2019-02-10,ZTS,9.999969e-01,2.544781e-06,5.711802e-07


In [ ]:
sentiment_result2019p1.to_csv('Dataset/sentiment2019p1.csv')

### Part 2

In [ ]:
#FinBERT Tokenizer and model

finbert19p2 = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')
tokenizer19p2 = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone',model_max_length=512)

In [ ]:
sp500_2019p2 =sp500_year[(sp500_year['date'] > pd.to_datetime("2019-06-01")) & (sp500_year['date'] <= pd.to_datetime("2020-01-01"))]

with open('sp500_2019part2.pkl','wb') as sp500_2019_pkl2:
  pickle.dump(sp500_2019p2,sp500_2019_pkl2)

sp500_2019p2

,title,date,stock
70,stocks that hit 52week highs on friday,2019-12-27,A
71,stocks that hit 52week highs on monday,2019-12-23,A
72,stocks that hit 52week highs on friday,2019-12-20,A
73,stocks that hit 52week highs on thursday,2019-12-12,A
74,theres a new trading tool that allows traders ...,2019-12-11,A
...,...,...,...
401856,barclays initiates coverage on zoetis with ove...,2019-06-11,ZTS
401857,the daily biotech pulse takeda pulls plug on a...,2019-06-06,ZTS
401858,the daily biotech pulse curtains down on asco ...,2019-06-05,ZTS
401859,shares of several healthcare companies are tra...,2019-06-04,ZTS


In [ ]:
with open('sp500_2019part2.pkl','rb') as loading3:
  sp500_2019p2 = pickle.load(loading3)

In [ ]:
sp500_2019p2

In [ ]:
headline_list2019p2 = sp500_2019p2['title'].tolist()
stock_column2019p2 = sp500_2019p2['stock'].tolist()
date_column2019p2 = sp500_2019p2['date'].tolist()

In [ ]:
nlp2019p2 = pipeline("text-classification", model=finbert19p2, tokenizer=tokenizer19p2)

In [ ]:
results2019p2 = nlp2019p2(headline_list2019p2)

with open ('res2019p2.pkl', 'wb') as files2:
  pickle.dump(results2019p2,files2)

In [ ]:
results2019p2

In [ ]:
with open('res2019p2.pkl','rb') as loading4:
  result_2019p2 = pickle.load(loading4)

In [ ]:
sentiment_result2019p2 = []

def chunk_list(lst, n):
    "Yield successive n-sized chunks from lst."
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


STRIDE = 70

finbert19p2.eval()

n=0
for lines, stocks, dates in zip(chunk_list(headline_list2019p2, STRIDE), chunk_list(stock_column2019p2, STRIDE),chunk_list(date_column2019p2, STRIDE) ):

  input = tokenizer19p2(lines, padding = True, truncation = True,  return_tensors='pt')

  outputs = finbert19p2(**input)

  with open ("Dataset/sentiment 2019 pickle/{f'output2_{n+1}.pkl}", 'wb') as result:
    pickle.dump(outputs,result)

  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)

  with open ("Dataset/sentiment 2019 pickle/f'pred2_{n+1}.pkl", 'wb') as result2:
    pickle.dump(prediction,result2)

  print(f"{n+1}/{int(len(headline_list2019p2)/STRIDE)}")

  for headline, dates, stock, neu, pos, neg in zip(lines, dates, stocks, prediction[:, 0].tolist(), prediction[:, 1].tolist(), prediction[:, 2].tolist() ):
    # sentiment_result = sentiment_result.append({'Headline': headline, 'Date': dates, 'Stock': stock, 'Neu': neu, 'Pos': pos, 'Neg': neg}, ignore_index=True)
    sentiment_result2019p2.append([headline, dates, stock, neu, pos, neg])



  n+=1





In [ ]:
sentiment_result2019p2 = pd.DataFrame(sentiment_result2019p2, columns= ['Headline', 'Date', 'Stock', 'Neutral', 'Positive', 'Negative' ])

sentiment_result2019p2

,Headline,Date,Stock,Neutral,Positive,Negative
0,stocks that hit 52week highs on friday,2019-12-27,A,9.964143e-01,1.911975e-05,3.566539e-03
1,stocks that hit 52week highs on monday,2019-12-23,A,9.972605e-01,1.044759e-05,2.729112e-03
2,stocks that hit 52week highs on friday,2019-12-20,A,9.964143e-01,1.911975e-05,3.566539e-03
3,stocks that hit 52week highs on thursday,2019-12-12,A,9.981936e-01,1.231802e-05,1.794033e-03
4,theres a new trading tool that allows traders ...,2019-12-11,A,9.983510e-01,1.627752e-03,2.131616e-05
...,...,...,...,...,...,...
27169,barclays initiates coverage on zoetis with ove...,2019-06-11,ZTS,7.076375e-06,9.999930e-01,1.040078e-08
27170,the daily biotech pulse takeda pulls plug on a...,2019-06-06,ZTS,9.999253e-01,5.453702e-05,2.018459e-05
27171,the daily biotech pulse curtains down on asco ...,2019-06-05,ZTS,9.999394e-01,8.320408e-07,5.975598e-05
27172,shares of several healthcare companies are tra...,2019-06-04,ZTS,3.552455e-08,1.000000e+00,3.186691e-08


In [ ]:
sentiment_result2019p2.to_csv('Dataset/sentiment2019p2.csv')

## 2020

In [ ]:
#FinBERT Tokenizer and model

finbert20 = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')
tokenizer20 = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone',model_max_length=512)

In [ ]:
sp500_2020 =sp500_year[sp500_year['date']>=pd.to_datetime("2020-01-01")]


sp500_2020

,title,date,stock
0,stocks that hit 52week highs on friday,2020-06-05,A
1,stocks that hit 52week highs on wednesday,2020-06-03,A
2,71 biggest movers from friday,2020-05-26,A
3,46 stocks moving in fridays midday session,2020-05-22,A
4,b of a securities maintains neutral on agilent...,2020-05-22,A
...,...,...,...
401796,raymond james initiates coverage on zoetis wit...,2020-01-10,ZTS
401797,the daily biotech pulse positive data readouts...,2020-01-10,ZTS
401798,stocks that hit 52week highs on thursday,2020-01-09,ZTS
401799,the daily biotech pulse mercks keytruda snags ...,2020-01-09,ZTS


In [ ]:
headline_list2020 = sp500_2020['title'].tolist()
stock_column2020 = sp500_2020['stock'].tolist()
date_column2020 = sp500_2020['date'].tolist()

In [ ]:
nlp2020 = pipeline("text-classification", model=finbert20, tokenizer=tokenizer20)

In [ ]:
results2020 = nlp2020(headline_list2020)

In [ ]:
results2020

[{'label': 'Neutral', 'score': 0.996414303779602},
 {'label': 'Neutral', 'score': 0.9977374076843262},
 {'label': 'Neutral', 'score': 0.9972858428955078},
 {'label': 'Neutral', 'score': 0.9999960660934448},
 {'label': 'Positive', 'score': 0.8801947236061096},
 {'label': 'Negative', 'score': 0.9998061060905457},
 {'label': 'Negative', 'score': 0.8813025951385498},
 {'label': 'Positive', 'score': 1.0},
 {'label': 'Positive', 'score': 0.9999995231628418},
 {'label': 'Neutral', 'score': 0.9999744892120361},
 {'label': 'Neutral', 'score': 0.9999980926513672},
 {'label': 'Positive', 'score': 0.9999998807907104},
 {'label': 'Neutral', 'score': 0.9999990463256836},
 {'label': 'Positive', 'score': 1.0},
 {'label': 'Positive', 'score': 1.0},
 {'label': 'Neutral', 'score': 0.9999940395355225},
 {'label': 'Positive', 'score': 0.9999480247497559},
 {'label': 'Neutral', 'score': 0.9999309778213501},
 {'label': 'Neutral', 'score': 0.9998477697372437},
 {'label': 'Positive', 'score': 0.999282896518707

In [ ]:
sentiment_result2020 = []

def chunk_list(lst, n):
    "Yield successive n-sized chunks from lst."
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


STRIDE = 100

finbert20.eval()

n=0
for lines, stocks, dates in zip(chunk_list(headline_list2020, STRIDE), chunk_list(stock_column2020, STRIDE),chunk_list(date_column2020, STRIDE) ):

  input = tokenizer20(lines, padding = True, truncation = True,  return_tensors='pt')

  outputs = finbert20(**input)

  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)

  print(f"{n+1}/{int(len(headline_list2020)/STRIDE)}")

  for headline, dates, stock, neu, pos, neg in zip(lines, dates, stocks, prediction[:, 0].tolist(), prediction[:, 1].tolist(), prediction[:, 2].tolist() ):
    # sentiment_result = sentiment_result.append({'Headline': headline, 'Date': dates, 'Stock': stock, 'Neu': neu, 'Pos': pos, 'Neg': neg}, ignore_index=True)
    sentiment_result2020.append([headline, dates, stock, neu, pos, neg])

  n+=1

1/332
2/332
3/332
4/332
5/332
6/332
7/332


In [ ]:
sentiment_result2020 = pd.DataFrame(sentiment_result2020, columns= ['Headline', 'Date', 'Stock', 'Neutral', 'Positive', 'Negative' ])

sentiment_result2020

In [ ]:
sentiment_result2020.to_csv('Dataset/sentiment2020.csv')